> First time use: follow instructions in the README.md file in this directory.



# Receitas

Várias receitas para usar Timelink com notebooks

[Sumário]

---

# Receipts

Receipts for using Timelink with notebooks

[summary]


---
## Setup

### Basic setup

Create a `TimelinkNotebook` instance.

This will provide an interface to the 
various `Timelink` functions.



In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


### Changing the default values

When called with no parameters `TimelinkNotebook` 
assumes a certain directory layout containing the notebook
and from that layout autoconfigures various parameters.

The default layout:

* **project-directory**
    * **database** (database related filed)
    * **notebooks**  _(directory with the current notebook)_ 
    * **kleio**  (source files in kleio format)
    * **identifications** (files with identification information)

Based on this layout `TimelinkNotebook` assumes a few default values:

* **database name**
    * the name of the *project directory* is used as the **database name**, sanitized to produce a valid database name
      (hifens and spaces are replaced with _ ).
* **database type**
  * the type of the database is assumed to be **sqlite**  and a **sqlite** database is created in 
      *database/sqlite*
    * the `db_type` parameter can be used to set the database to `postgres`. 
* **Kleio server home**, working directory for the _Kleio server_ is assumed to be the **project directory**
  * The _Kleio server_ is an application
    that processes transcriptions of historical _Kleio_ notation and generates data in a format
    that can be imported to the database.

These default values can be changed with parameters when creating the `TimelinkNotebook`  instance:

```python
tlnb = TimelinkNotebook(
    project_name="my-project",
    project_home="~/projects/m-project",
    db_type="postgres",
    db_name="my_project_db",
    kleio_image="timelinkserver/kleio-server",
    kleio_version="12.0.0",
    postgres_image="postgres",
    postgres_version="latest",
    sqlite_dir="~/databases",
    sql_echo=True
)
```

### Examining the current configuration

In [2]:

tlnb.print_info()


Timelink version: 1.1.15
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: WO2v6...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: priceless_darwin
Kleio version requested: latest
Kleio server version: 12.6.577 (2024-10-24 16:53:53)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


### Listing available databases

In [3]:
pd = tlnb.get_postgres_databases()
print(f"Postgres databases: {pd}")
sd = tlnb.get_sqlite_databases()
print(f"Sqlite databases: {sd}")


Postgres databases: ['timelink', 'tests', 'rentities', 'xsameas', 'tests_users', 'test_project']
Sqlite databases: ['../database/sqlite/timelink.sqlite', '../database/sqlite/dehergne_locations.sqlite']


### Database status (table row count)

In [4]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,alembic_version,1
2,aregisters,1
3,attributes,26075
4,blinks,134
5,class_attributes,70
6,classes,14
7,entities,32281
8,geoentities,359
9,goods,0


## Dealing with kleio files

### List Kleio files available

In [5]:
kleio_files = tlnb.get_kleio_files()
kleio_files.head()

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-0-abrev.cli,dehergne-0-abrev.cli,2024-12-21 13:36:32.775587+00:00,V,2024-12-21 13:36:00+00:00,0,0,I,0,0,No errors,No warnings,2024-12-21 13:37:14.837608,/rest/reports/sources/dehergne-0-abrev.rpt,/rest/exports/sources/dehergne-0-abrev.xml
1,sources/dehergne-a.cli,dehergne-a.cli,2024-12-21 13:36:19.502779+00:00,V,2024-12-21 13:36:00+00:00,0,0,I,0,0,No errors,No warnings,2024-12-21 13:49:50.504188,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
2,sources/dehergne-b.cli,dehergne-b.cli,2024-12-21 13:36:39.486670+00:00,V,2024-12-21 13:36:00+00:00,0,0,I,0,0,No errors,No warnings,2024-12-21 13:49:59.678463,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
3,sources/dehergne-c.cli,dehergne-c.cli,2025-01-04 04:37:01.794996+00:00,V,2025-01-04 04:37:00+00:00,0,0,I,0,0,No errors,No warnings,2025-01-04 04:37:17.815541,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
4,sources/dehergne-d.cli,dehergne-d.cli,2024-12-21 13:36:38.127976+00:00,V,2024-12-21 13:36:00+00:00,0,0,I,0,0,No errors,No warnings,2024-12-21 13:50:05.016281,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml


Show only translation and import status

In [6]:
kleio_files[["name","status","import_status"]]

,name,status,import_status
0,dehergne-0-abrev.cli,V,I
1,dehergne-a.cli,V,I
2,dehergne-b.cli,V,I
3,dehergne-c.cli,V,I
4,dehergne-d.cli,V,I
5,dehergne-e.cli,V,I
6,dehergne-f.cli,V,I
7,dehergne-g.cli,V,I
8,dehergne-h.cli,V,I
9,dehergne-i.cli,V,I


Show translation and import errors

In [7]:
kleio_files[["name","errors","warnings","import_errors","import_warnings"]]

,name,errors,warnings,import_errors,import_warnings
0,dehergne-0-abrev.cli,0,0,0,0
1,dehergne-a.cli,0,0,0,0
2,dehergne-b.cli,0,0,0,0
3,dehergne-c.cli,0,0,0,0
4,dehergne-d.cli,0,0,0,0
5,dehergne-e.cli,0,0,0,0
6,dehergne-f.cli,0,0,0,0
7,dehergne-g.cli,0,0,0,0
8,dehergne-h.cli,0,0,0,0
9,dehergne-i.cli,0,0,0,0


### Translation and Import reports

Translation reports.

Pass a dataframe with Kleio files and a row number to get the translation report

In [8]:
rpt=tlnb.get_translation_report(kleio_files,rows=1)
print(rpt)

KleioTranslator - server version 12.6 - build 577 2024-10-24 16:53:53
21-12-2024 13-36

Processing data file dehergne-a.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 233
Obs: 
** Processing source fonte$dehergne-a
18: lista$dehergne-notices-a
*** End of File

Line 576 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-belchior-miguel-carneiro-leitao) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 776 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-andre-palmeiro) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 781 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-antonio-francisco-cardim) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 940 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-jean-regis-lieou) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 1002 "SAME AS" TO EX

Or use the file name

In [9]:
file = kleio_files.iloc[0].rpt_url
print(file)
rpt=tlnb.get_translation_report(file)
print(rpt)

/rest/reports/sources/dehergne-0-abrev.rpt
KleioTranslator - server version 12.6 - build 577 2024-10-24 16:53:53
21-12-2024 13-36

Processing data file dehergne-0-abrev.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: 
Prefix: 
Autorel: 
Translation count: 4
Obs: 
** Processing source fonte$dehergne-0-abrev
*** End of File


Structure file: /kleio-home/structures/sources.str
Structure processing report: /kleio-home/structures/sources.srpt
Structure in JSON: /kleio-home/structures/sources.str.json

Kleio file: /kleio-home/sources/dehergne-0-abrev.cli
Original file: /kleio-home/sources/dehergne-0-abrev.org
Previous version: /kleio-home/sources/dehergne-0-abrev.old
Temp file with ids: /kleio-home/sources/dehergne-0-abrev.ids
0  errors. 
0  warnings.
Translation finished.



Import report

In [10]:
rpt = tlnb.get_import_rpt(kleio_files,rows=1)
print(rpt)

dehergne-a.cli
No errors
No warnings





# Atualizar base de dados

Atualiza traduções de fontes e importa quando tradução não tem erros.

---

# Update database

Updates source translations and imports into database sources with no errors.

In [11]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()

Show imported files status

In [12]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                31 non-null     object             
 1   name                31 non-null     object             
 2   size                31 non-null     int64              
 3   directory           31 non-null     object             
 4   modified            31 non-null     datetime64[ns, UTC]
 5   modified_iso        31 non-null     datetime64[ns, UTC]
 6   modified_string     31 non-null     object             
 7   qtime               31 non-null     datetime64[ns, UTC]
 8   qtime_string        31 non-null     object             
 9   source_url          31 non-null     object             
 10  status              31 non-null     object             
 11  translated          31 non-null     datetime64[ns, UTC]
 12  translated_string   31 non-null     ob

Check the import status of the translated files
```python
    I = "I" # imported
    E = "E" # imported with error
    W = "W" # imported with warnings no errors
    N = "N" # not imported
    U = "U" # translation updated need to reimport

``` 

In [13]:
imported_files_df[["import_status","import_errors","import_warnings","name","imported","path"]].sort_values("name")

,import_status,import_errors,import_warnings,name,imported,path
0,I,0,0,dehergne-0-abrev.cli,2024-12-21 13:37:14.837608,sources/dehergne-0-abrev.cli
1,I,0,0,dehergne-a.cli,2024-12-21 13:49:50.504188,sources/dehergne-a.cli
2,I,0,0,dehergne-b.cli,2024-12-21 13:49:59.678463,sources/dehergne-b.cli
3,I,0,0,dehergne-c.cli,2025-01-04 04:37:17.815541,sources/dehergne-c.cli
4,I,0,0,dehergne-d.cli,2024-12-21 13:50:05.016281,sources/dehergne-d.cli
5,I,0,0,dehergne-e.cli,2024-12-21 13:50:06.540929,sources/dehergne-e.cli
6,I,0,0,dehergne-f.cli,2024-12-21 13:50:14.071841,sources/dehergne-f.cli
7,I,0,0,dehergne-g.cli,2024-12-21 13:37:51.862883,sources/dehergne-g.cli
8,I,0,0,dehergne-h.cli,2024-12-21 13:50:16.801054,sources/dehergne-h.cli
9,I,0,0,dehergne-i.cli,2024-12-21 13:37:55.552081,sources/dehergne-i.cli


## Todo

This as data frame in a single function
* TimelinkNotebook.translate([files_df,rows=List[int],status="T"])
* TimelinkNotebook.import(files_df, rows=List[int])



# Obter dados
---

# Getting data

### Procurar pessoa

---
### Search for people

#### Search persons by name

In [14]:
from timelink.api.models import Person
from sqlalchemy import select

show_only=10

with tlnb.db.session() as session:
    stmt = select(Person).where(Person.name.like('% Valignano%'))
    print(stmt)
    persons = session.execute(stmt).scalars().all()
    print()
    for person in persons[:show_only]:
        print(person.id, person.name,person.sex)


SELECT persons.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.name LIKE :name_1

deh-duarte-de-sande Duarte de Sande m


### Search other Entities

#### get the Entity classes in the database

In [15]:
from sqlalchemy import select, func
from timelink.api.models import Entity

models = tlnb.db.get_models_ids()
models

['attribute',
 'relation',
 'act',
 'source',
 'person',
 'good',
 'object',
 'geoentity',
 'rgeoentity',
 'robject',
 'rperson',
 'rentity',
 'class',
 'aregister',
 'entity']

#### Get columns of an entity type

In [16]:
table = tlnb.db.get_table("entity")
print(table.name)
list(table.columns)

entities


[Column('id', String(), table=<entities>, primary_key=True, nullable=False),
 Column('class', String(), table=<entities>, nullable=False),
 Column('inside', String(), ForeignKey('entities.id'), table=<entities>),
 Column('the_order', Integer(), table=<entities>),
 Column('the_level', Integer(), table=<entities>),
 Column('the_line', Integer(), table=<entities>),
 Column('groupname', String(), table=<entities>),
 Column('updated', DateTime(), table=<entities>, default=CallableColumnDefault(<function datetime.utcnow at 0x117ec8670>)),
 Column('indexed', DateTime(), table=<entities>)]

#### Search any entity type

## IMPROVE

In [17]:
from timelink.api.models import Entity
from sqlalchemy import select, func

Geoentity = tlnb.db.get_model("geoentity")
stmt = select(Geoentity).where(Geoentity.name.like('H%'))
print(stmt)
with tlnb.db.session() as session:

    result = session.execute(stmt).scalars().all()
    for row in result[:4]:
        print()
        print(row.the_type,row.name,row.id,row.obs)
        print(row.to_kleio())

SELECT geoentities.id, entities.id AS id_1, entities.class, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, geoentities.name, geoentities.the_type, geoentities.obs 
FROM entities JOIN geoentities ON entities.id = geoentities.id 
WHERE geoentities.name LIKE :name_1

geo2 Hangchou deh-r1701-hangchou extra_info: {"name": {"comment": "Hangzhou, @wikidata:Q4970"}}
geo2$Hangchou/geo2/id=deh-r1701-hangchou/obs="""extra_info: {"name": {"comment": "Hangzhou, @wikidata:Q4970"}}"""
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q4970"#Hangzhou, @wikidata:Q4970%Q4970/17010000
  atr$activa/sim/1701
  atr$residencia-missao/Jesuíta/1701
  atr$activa/não/1701
  atr$residencia-missao/Franciscanos/1701
  geo3$Haining/geo3/id=deh-r1701-hainang
    atr$activa/não/1701

geo3 Haining deh-r1701-hainang 
geo3$Haining/geo3/id=deh-r1701-hainang
  atr$activa/não/1701

geo2 Hinghwa deh-r1701-hinghwa extra_info: {"nam

#### Get person by id

Show a single person or entity in Kleio notation

In [18]:
from timelink.api.models.person import Person

id = 'deh-duarte-de-sande'
with tlnb.db.session() as session:
    # get Person with id
    p = session.get(Person, id)
    print(p)

n$Duarte de Sande/m/id=deh-duarte-de-sande
  ls$nacionalidade/Portugal/0
  ls$jesuita-estatuto/Padre/0
  ls$nome/Edoardo de Sande/0
  ls$nome-chines/Mong San-Tö Ning-Houan/0
  ls$nascimento@wikidata/"https://www.wikidata.org/wiki/Q170903"%Guimarães/15471025
  ls$nascimento/Guimarães#@wikidata:Q170903/15471025
  ls$jesuita-entrada@wikidata/"https://www.wikidata.org/wiki/Q597"%Lisboa/15620600
  ls$jesuita-entrada/Lisboa#@wikidata:Q597/15620600
  ls$idade/15/15620600
  ls$jesuita-cargo/Superior dos Jesuítas/15780324
  ls$embarque/S. Luís#para a Índia/15780324
  ls$wicky/196/15780324
  ls$wicky-viagem/25/15780324
  ls$estadia@wikidata/"https://www.wikidata.org/wiki/Q1949022"%Baçaim/0
  ls$estadia/Baçaim#@wikidata:Q1949022/0
  ls$jesuita-cargo/Reitor em Baçaim%recteur/0
  ls$estadia@wikidata/"https://www.wikidata.org/wiki/Q1171"%Goa/15850501
  ls$estadia/Goa#@wikidata:Q1171/15850501
  ls$partida@wikidata/"https://www.wikidata.org/wiki/Q14773"%Macau/15850501
  ls$partida/Macau#@wikidata:Q147

### Show other type of entities by id in Kleio

In [23]:
from timelink.api.models import Entity

with tlnb.db.session() as session:
    # get Entity with id
    ent = tlnb.db.get_entity(id="deh-r1644-chekiang", session=session)
    print(ent.to_kleio())

TypeError: Entity.get_entity() got multiple values for argument 'id'



###  Obter um dataframe a partir de atributos

---


###  Get a Dataframe from attributes


#### Exemplo: Faculdade, data de entrada e data de saída e grau dos naturais de Coimbra

In [ ]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='person',
                    the_type='naturalidade',
                    the_value='Coimbra',
                    more_info=['name','sex'],
                    name_like='% Aboim',
                    more_cols=['faculdade','uc.entrada','uc.saida'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 140349 to 140367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               2 non-null      object
 1   sex                2 non-null      object
 2   naturalidade       2 non-null      object
 3   naturalidade.date  2 non-null      object
 4   naturalidade.obs   0 non-null      object
 5   faculdade          2 non-null      object
 6   faculdade.date     2 non-null      object
 7   faculdade.obs      0 non-null      object
 8   uc.entrada         2 non-null      object
 9   uc.entrada.date    2 non-null      object
 10  uc.entrada.obs     0 non-null      object
 11  uc.saida           2 non-null      object
 12  uc.saida.date      2 non-null      object
 13  uc.saida.obs       0 non-null      object
dtypes: object(14)
memory usage: 348.0+ bytes


In [21]:
df.head(5)

,name,sex,naturalidade,naturalidade.date,naturalidade.obs,faculdade,faculdade.date,faculdade.obs,uc.entrada,uc.entrada.date,uc.entrada.obs,uc.saida,uc.saida.date,uc.saida.obs
id,,,,,,,,,,,,,,
140349,António de Aboim,m,Coimbra,1566-12-20,None,Cânones,1566-12-20,None,1566-12-20,1566-12-20,None,1574-07-24,1574-07-24,None
140367,Manuel de Vargas de Aboim,m,Coimbra,20200211,None,Cânones,20200211,None,0000-00-00,20200211,None,0000-00-00,20200211,None


#### obter attributos de outras entidades

In [22]:
from timelink.pandas import entities_with_attribute


# Get list of people with with a certain value in a specific attribute
df = entities_with_attribute(
                    entity_type='geoentity',
                    more_info=['name'],
                    the_type='activa',
                    the_value='sim',
                    more_cols=['residencia-missao'],
                    db=tlnb.db,
                    sql_echo=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-anhai to deh-r1644-yunnan
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    165 non-null    object
 1   activa                  165 non-null    object
 2   activa.date             165 non-null    object
 3   activa.obs              4 non-null      object
 4   residencia-missao       38 non-null     object
 5   residencia-missao.date  38 non-null     object
 6   residencia-missao.obs   4 non-null      object
dtypes: object(7)
memory usage: 10.3+ KB


In [23]:
df.head(30)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None
deh-r1644-chala,Chala,sim,1610,None,NaN,NaN,NaN
deh-r1644-changchow-fou,Changchow,sim,1643,None,NaN,NaN,NaN
deh-r1644-changshu,Changshu,sim,1623,None,Jesuíta,1635,"R 1635 «Cham Xo», cf AHSI 28 (1951) 311-312"
deh-r1644-chengting,Chengting,sim,1621,None,NaN,NaN,NaN




###  Remover colunas sem valores

---



###  Remove empty columns

In [24]:
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, deh-r1644-anhai to deh-r1644-yunnan
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   name                    165 non-null    object
 1   activa                  165 non-null    object
 2   activa.date             165 non-null    object
 3   activa.obs              4 non-null      object
 4   residencia-missao       38 non-null     object
 5   residencia-missao.date  38 non-null     object
 6   residencia-missao.obs   4 non-null      object
dtypes: object(7)
memory usage: 10.3+ KB


In [25]:
df.head(5)

,name,activa,activa.date,activa.obs,residencia-missao,residencia-missao.date,residencia-missao.obs
id,,,,,,,
deh-r1644-anhai,Anhai,sim,1634,None,NaN,NaN,NaN
deh-r1644-bankao,Bankao,sim,1636,None,NaN,NaN,NaN
deh-r1644-cantao,Cantão,sim,1555,None,Jesuíta,1580,None
deh-r1644-cantao,Cantão,sim,1555,None,Franciscanos,0000,None
deh-r1644-cantao,Cantão,sim,1555,None,Dominicanos,0000,None


In [26]:
ids = df.index.unique()
for id in ids[:5]:
    ent = tlnb.db.get_entity(id)
    print(ent.to_kleio())
    print()

geo3$deh-r1644-anhai/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1634

geo3$deh-r1644-bankao/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1636

geo2$deh-r1644-cantao/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/1555
  atr$residencia-missao/Jesuíta/1580
  atr$residencia-missao/Franciscanos/0000
  atr$residencia-missao/Dominicanos/0000
  geo3$deh-r1644-tungkun/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
  geo3$deh-r1644-quon-yao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1621
  geo3$deh-r1644-lampacao/type=geoentity
    rel$function-in-act/geo3/deh-chre-1644/16440000
    atr$activa/sim/1535

geo3$deh-r1644-chala/type=geoentity
  rel$function-in-act/geo3/deh-chre-1644/16440000
  atr$activa/sim/1610

geo2$deh-r1644-changchow-fou/type=geoentity
  rel$function-in-act/geo2/deh-chre-1644/16440000
  atr$activa/sim/1643



## Contagens

---

## Counting



###  Contagem de atributos a partir de uma tabela em memória

---

###  Count attributes from an existing dataframe



In [27]:
# create a column with the index values which are the id numbers
df['id'] = df.index.values

col = 'faculdade' # subotal by this column

# Use pandas groupby and specify unique value count for id
df_totals = df.groupby(col).agg({'id':'nunique',
                                                  'uc.entrada':'min',
                                                  'uc.saida':'max'})

df_totals.sort_values('id',ascending= False).head(30)

KeyError: 'faculdade'


### Contagens na base de dados

Quando o atributo tem muitos valores e não é necessário
ter todas as pessoas em memória: contagem feita na base de dados

---

### Counting directly in the database
When there are many values and it is not
necessary to have all the people in memory:
count directly in the database.




In [ ]:
from timelink.pandas import attribute_values

df_totals = attribute_values('grau',db=tlnb.db)


In [ ]:
df_totals.head(10)


,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


#### Filtrar por datas

Para evitar remissivas com data zero

---

#### Filter by dates

Avoid cross-references with zero date

##ERROR

In [ ]:
df_totals = attribute_values('grau',dates_between=('1535','1919'),db=tlnb.db)

In [ ]:
df_totals.head(10)

,count,date_min,date_max
value,,,
Bacharel,170,1554-07-19,1912-08-03
Formatura,153,1574-07-24,1905-06-19
Licenciado,33,1574-06-03,1886-02-27
Bacharel em Artes,25,1574-03-14,1766-07-19
Doutor,11,1560-12-22,1887-11-27
Licenciado em Artes,5,1574-05-15,1738-06-17
Mestre,2,1710-10-05,1768-10-23


## Visualizar registos

---

## View records





### Visualizar uma pessoa

---

### View a person


#### Atributos de uma pessoa numa tabela, uma linha por attributo

---

#### Person attributes in a dataframe, one line per attribute

In [3]:
import pandas as pd
from timelink.pandas import group_attributes as person_attributes

pd.set_option('display.max_rows',1000)

id = 'deh-duarte-de-sande'
pdf = person_attributes([id],db=tlnb.db)  # note id in a list
pdf.info()
pdf[['the_date','the_type','the_value','attr_obs']].sort_values(['the_date','the_type'])

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, deh-duarte-de-sande to deh-duarte-de-sande
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   the_type   44 non-null     object
 1   the_value  44 non-null     object
 2   the_date   44 non-null     object
 3   attr_obs   44 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


,the_date,the_type,the_value,attr_obs
id,,,,
deh-duarte-de-sande,0,dehergne,741,"""""""\n Sande, Duarte (Edoardo) de (p..."
deh-duarte-de-sande,0,dehergne@archive,https://archive.org/details/bhsi37/page/276/mo...,"extra_info: {""value"": {""original"": ""741""}}"
deh-duarte-de-sande,0,estadia,Baçaim,"extra_info: {""value"": {""comment"": ""@wikidata:Q..."
deh-duarte-de-sande,0,estadia@wikidata,https://www.wikidata.org/wiki/Q1949022,"extra_info: {""value"": {""original"": ""Ba\u00e7ai..."
deh-duarte-de-sande,0,jesuita-cargo,Reitor em Baçaim,"extra_info: {""value"": {""original"": ""recteur""}}"
deh-duarte-de-sande,0,jesuita-estatuto,Padre,
deh-duarte-de-sande,0,nacionalidade,Portugal,
deh-duarte-de-sande,0,nome,Edoardo de Sande,
deh-duarte-de-sande,0,nome-chines,Mong San-Tö Ning-Houan,


#### Atributos de uma pessoa numa tabela, attributos em colunas

---

#### Person attributes in a dataframe, attributes in columns

In [30]:
# Get list of people with with a certain value in a specific attribute

df = entities_with_attribute(
                    entity_type='person',
                    the_type='uc.entrada',  # we need a base attribute
                    more_info=['name','sex'],
                    more_cols=['instituta','matricula.faculdade','matricula.ano'],
                    filter_by=[140349],
                    db=tlnb.db,
                    sql_echo=False)
view_cols = ['name','matricula.ano.date','matricula.ano','matricula.ano.obs']
df[view_cols].sort_values('matricula.ano.date')

,name,matricula.ano.date,matricula.ano,matricula.ano.obs
id,,,,
140349,António de Aboim,1571-07-20,Cânones.1571,20.07.1571
140349,António de Aboim,1573-10-07,Cânones.1573,07.10.1573
140349,António de Aboim,1574-07-24,Cânones.1574,24.07.1574


#### Examinar potenciais duplicados

---

#### Examine potentital duplicates

In [15]:
from timelink.pandas import display_group_attributes
pd.set_option('display.max_rows',250)

no_show=['código-de-referência','data-do-registo','url','faculdade.ano','naturalidade.ano',
         'matricula-faculdade.ano','nome-apelido','nome-primeiro','nome-geografico.ano',
         'grau.ano','matricula-outra.ano','nome-geografico','instituta.ano']

dup_ids = ['234295','234710',]  # Alexandre Metelo de

display_group_attributes(dup_ids,
                             header_cols=['uc-entrada','naturalidade','faculdade','nome-pai'],
                             exclude_attributes=no_show,
                             sort_attributes=['date','type','value'],
                             cmap_name='Pastel1')

,id,uc-entrada,naturalidade,faculdade,nome-pai
0,234295,1704-11-07,Marialva,Cânones,nan
1,234710,1705-10-24,Marialva,Matemática,Manuel Cardoso Metelo


,date,id,type,value,attr_obs
0,1704-11-07,234295,faculdade,Cânones,Cânones
1,1704-11-07,234295,instituta,1704-11-07,07.11.1704 1704-11-07
2,1704-11-07,234295,naturalidade,Marialva,None
3,1704-11-07,234295,nome,Alexandre Metelo de Sousa,None
4,1704-11-07,234295,uc-entrada,1704-11-07,None
5,1704-11-07,234295,uc-entrada.ano,1704,None
6,1705-10-24,234710,faculdade,Matemática,Matemática
7,1705-10-24,234295,matricula-faculdade,Cânones,24.10.1705
8,1705-10-24,234710,matricula-faculdade,Matemática,24.10.1705
9,1705-10-24,234710,naturalidade,Marialva,None


#### Notação Kleio

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation

See [Kleio notation](README_kleio.md)

#### Notação Kleio directamente da base de dados

Ver [Kleio notation](README_kleio.md) [EN]

---

#### Kleio notation directly from database

See [Kleio notation](README_kleio.md)

In [33]:
from timelink.mhk.models.person import Person

with tlnb.db.session() as session:

    p: Person = session.query(Person).order_by(Person.id).first()
    k = p.to_kleio()
    print(p.to_kleio())


n$António Pinto Abadeço/m/id=140337/obs="""
      """

                  Id: 140337
                  Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/A/000001

                  Nome        : António Pinto Abadeço
                  Data inicial: 1705-10-01
                  Data final  : 1710-10-01
                  Filiação: António Pinto

                  Naturalidade: Abrantes
                  Faculdade: Cânones

                  Matrícula(s): 01.10.1705
                  01.10.1706
                  01.01.1707
                  01.10.1708
                  01.10.1709
                  01.10.1710
                  Instituta:
                  Bacharel: 28.06.1709
                  Formatura:
                  Licenciado:
                  Doutor:

                  Outras informações:
              """
  """
  rel$function-in-act/n/António Pinto Abadeço/auc-alumni-A-140337-140771/20200211
  atr$código-de-referência/""PT/AUC/ELU/UC-AUC/B/001-001/A/000001""/2021-05-17
  atr$data-